In [ ]:
from __future__ import print_function, division

# Images in Astronomy

In this lesson we are going to look at aspects of processing and viewing images specific to Astronomy and Solar Astronomy. By the end of this lesson you should understand:

* Projected Coordinate Systems in Images
* World Coordinate Systems
* Using WCS to calculate coordinates in images
* Plotting images with WCS in images
* Using SunPy Map

## Projected Coordinate Systems

<div style="float:left; width:39%">
<p>
When taking images of the sky, we are projecting the spherical celestial coordinate system onto a 2-dimensional plane, which means that there is no simple linear relation between pixel coordinates and celestial coordinates
</p>
<p>
There are multiple coordinate systems used to describe the locations in 2D and 3D space for both Astronomy and Solar Physics. We shall use a couple of these systems here as examples but if you want to know more about them there are many of resources avalible.
</p>
</div>
<div style="float:left; width:59%">
<img src="coord_inset.png" width=100% />
</div>

### Physical Coordinate Systems

Projected coordinate systems are one type of physical coordinate systems, they are the one we will be focusing on in this lesson due to their applicability to imaging data. Astropy and SunPy support representing point in many different physical coordinate systems, both projected and fully 3D, such as ICRS or Helioprojective.

In [1]:
import sunpy.coordinates
from astropy.coordinates import SkyCoord
import astropy.units as u

ImportError: No module named 'sunpy'

In [ ]:
icrs = SkyCoord(10*u.deg, 86*u.deg, frame='icrs')
icrs

In [ ]:
hpc = SkyCoord(100*u.arcsec, 700*u.arcsec, frame='helioprojective')
hpc

In [ ]:
hpc.transform_to('heliographic_stonyhurst')

### World Coordinate System

#### From pixels  to physical coordinates

The FITS files have a standard for describing the physical coordinate system associated with imaging data, this is called the world coordinate system or WCS, sometimes the specific FITS version of this is referred to as FITS-WCS.

There are multiple papers describing the FITS-WCS standard for various types of data, there is a list here: http://fits.gsfc.nasa.gov/fits_wcs.html

As you learned in the previous lesson we can load FITS files with Astropy. To demonstrate a simple example of a FITS file with FITS-WCS information in the header we shall use an image from SunPy:

In [ ]:
from sunpy.data.sample import AIA_171_IMAGE
from astropy.io import fits

hdulist = fits.open(AIA_171_IMAGE)
hdulist.verify('silentfix')
hdulist[0].header

As you can see there are lots of keys in this and most other real world FITS headers. The ones we need to understand for FITS-WCS are:

Reference Pixel and Coordinate:

In [ ]:
header = hdulist[0].header

print(header['CRVAL1'], header['CRVAL2'])
print(header['CRPIX1'], header['CRPIX2'])

Pixel resolution (at the reference pixel):

In [ ]:
print(header['CDELT1'], header['CDELT2'])

Rotation angle, in degress (at the reference pixel):

In [ ]:
print(header['CROTA2'])

Coordinate System and Projection:

In [ ]:
print(header['CTYPE1'], header['CTYPE2'])

<section class="objectives panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Keyword Extraction </h2>
</div>

Extract and print out the `TELESCOP` value from the header.

Next, extract the `WAVELNTH` and `WAVEUNIT` values, use these to construct an astropy Quantity object for the wavelength of this image.



#### Solution 

In [ ]:
header['TELESCOP']

In [ ]:
import astropy.units as u
u.Quantity(header['WAVELNTH'], unit=header['WAVEUNIT'])

We could now sit down and work out how to convert from a pixel coordinate to a physical coordinate described by this header (Helioprojective).

However, we can cheat and just use Astropy.

In [ ]:
from astropy.wcs import WCS

wcs = WCS(header)

We can convert from pixel to world coordinate:

In [ ]:
wcs.wcs_pix2world(((100, 100),), 0)

Or back again:

In [ ]:
wcs.wcs_world2pix([[  3.59725669e+02,  -2.74328093e-01]], 0)

The last parameter to the two above examples is the 'origin' parameter. It is a flag that tells WCS if you indexes should be 0-based (like numpy) or 1-based (like FITS).
Here we are using 0 as we want to convert to and from numpy indexes of the array.

<section class="objectives panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> How large is the image? </h2>
</div>
<br/>
To get a little practise using Astropy's WCS calculate the world coordinates of the following pixels:
<code>
[-500, 0]
[500, 500]
[0, 0]
</code>
<br/>
</section>

In [ ]:
print(wcs.wcs_pix2world(((-500, 0),), 1))
print(wcs.wcs_pix2world(((500, 500),), 1))
print(wcs.wcs_pix2world(((0, 0),), 1))

### WCS and SkyCoord

A common usecase for WCS + Coordinates is to store or transform from pixel coordinates to one or more different physical coordinates. Combining Astropy WCS and Coordinates makes this easy.

Assuming we have the WCS object we created from the FITS header above we can get an astropy Coordinate Frame:

In [ ]:
from astropy.wcs.utils import wcs_to_celestial_frame

In [ ]:
hpc_frame = wcs_to_celestial_frame(wcs)
hpc_frame

We can then use this when creating `SkyCoord` objects:

In [ ]:
SkyCoord(100*u.arcsec, -500*u.arcsec, frame=hpc_frame)

*Note: For solar data you should always use `sunpy.map` for this*

## Plotting with wcsaxes

In this section we are going to use the wcsaxes package to make WCS aware image plots.

In [ ]:
import wcsaxes

For this example we are going to use a Hubble image.

In [ ]:
from astropy.io import fits

hdulist = fits.open('./h_n4571_f555_mosaic.fits.gz')
hdulist

In [ ]:
wcs = WCS(hdulist[0].header)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
ax = plt.subplot(111, projection=wcs)
ax.imshow(hdulist[0].data, cmap='gray', vmax=1000, interpolation=None, origin='lower')

This image now has physcial labels in the native coordinate system of the image. We can see what the coordinate system and projection of this image is using the 'CTYPE' header entries we saw earlier.

In [ ]:
print(hdulist[0].header['CTYPE1'], hdulist[0].header['CTYPE2'])

We can tell that this is in the FK5 coordinate system by the presence of a 'equinox' entry in the header:

In [ ]:
hdulist[0].header['equinox']

<section class="objectives panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Add some labels </h2>
</div>
<br/>
Now we have a nice plot with physically meaningful ticks, we should label our axes.
<br/>

Add labels to the axes saying "Right Ascension [degrees]" and "Declination [degrees]"
<br/>

Also overlay a coordinate grid using:
<code>ax.coords.grid()</code>
Look up the documentation for this method to see what parameters you can specify.
<br/>
</section>

#### Solution

In [ ]:
ax = plt.subplot(111, projection=wcs)
ax.imshow(hdulist[0].data, cmap='gray', vmax=1000, interpolation=None, origin='lower')
ax.set_xlabel("Right Ascension [degrees]")
ax.set_ylabel("Declination [degrees]")
ax.coords.grid(color='white', alpha=0.5, linestyle='solid')

Now we have a nice plot, we can do a couple of things to plot.

### Overplotting in Pixel Coordinates

In [ ]:
ax = plt.subplot(111, projection=wcs)

ax.imshow(hdulist[0].data, cmap='gray', vmax=1000, interpolation=None, origin='lower')
ax.set_xlabel("Right Ascension [degrees]")
ax.set_ylabel("Declination [degrees]")
ax.coords.grid(color='white', alpha=0.5, linestyle='solid')

ax.plot(3000, 3000, 'o')

### Overplotting in World Coordinates

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=wcs)

ax.imshow(hdulist[0].data, cmap='gray', vmax=1000, interpolation=None, origin='lower')
ax.set_xlabel("Right Ascension [degrees]")
ax.set_ylabel("Declination [degrees]")
ax.coords.grid(color='white', alpha=0.5, linestyle='solid')

ax.set_autoscale_on(False)

ax.plot(3000, 3000, 'o')
# Overplot in FK5 in Degrees
ax.plot(189.25, 14.23, 'o', transform=ax.get_transform('fk5'))

<section class="objectives panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Overplot in a different coordinate system </h2>
</div>
<br/>
Using `ax.get_transform()` you can specify coordinates in any system that astropy coordinates can transform to, try overplotting a point in the `'galactic'` system.
<br/>

Also overlay a coordinate grid using:
<code>ax.coords.grid()</code>
Look up the <a href="http://wcsaxes.readthedocs.io/en/latest/overlaying_coordinate_systems.html">documentation</a> for this method to see what parameters you can specify.
<br/>
</section>

###  Overplotting Another Coordinate System

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=wcs)
ax.imshow(hdulist[0].data, cmap='gray', vmax=1000, interpolation=None, origin='lower')
ax.set_xlabel("Right Ascension [degrees]")
ax.set_ylabel("Declination [degrees]")
ax.coords.grid(color='white', alpha=0.5, linestyle='solid')

overlay = ax.get_coords_overlay('galactic')
overlay.grid(color='orange', alpha=1, linestyle='solid')
overlay['l'].set_axislabel("Galactic Longitude [degrees]")
overlay['b'].set_axislabel("Galactic Latitude [degrees]")


ax.plot(287.5, 76.65, 'o', transform=ax.get_transform('galactic'))

## SunPy Map

The SunPy Map class is a wrapper for solar images which makes some of the above opertations easier.

In [ ]:
import sunpy.map
from sunpy.data.sample import AIA_171_ROLL_IMAGE

amap = sunpy.map.Map(AIA_171_ROLL_IMAGE)
amap.peek()

### Coordinate Systems

The SunPy map will calculate some of the things we did earlier automatically, and add some extra information.

In [ ]:
amap.coordinate_system

In [ ]:
amap.coordinate_frame

In [ ]:
amap.wcs

### Plotting

SunPy map uses these attributes to do it's plotting with WCSAxes like we did manually earlier. We can therefore use the overplotting and grid techniques to improve the plot:

In [ ]:
import astropy.units as u

In [ ]:
fig = plt.figure()
amap = sunpy.map.Map(AIA_171_ROLL_IMAGE)

im = amap.plot()
ax = plt.gca()
ax.set_autoscale_on(False)

x = 500*u.arcsec
y = -300*u.arcsec

ax.plot(x.to(u.deg), y.to(u.deg), 'o', transform=ax.get_transform('world'))

overlay = ax.get_coords_overlay('heliographic_stonyhurst')
overlay.grid(color='cyan', alpha=1, linestyle='solid')

In [ ]:
amap.pixel_to_data(100*u.pix, 200*u.pix)

In [ ]:
amap.data_to_pixel(0*u.arcsec, 0*u.arcsec)

<section class="objectives panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Rotate your Owl </h2>
</div>
Why is the Sun wonky?

Use the [`rotate()`](http://docs.sunpy.org/en/stable/code_ref/map.html#sunpy.map.mapbase.GenericMap.rotate) method of SunPy Map to align the coordinate grid to the pixel grid in this sample image.

Once you have run rotate, plot the resulting image, and compare with the one above.
</section>

In [ ]:
mr = amap.rotate()
mr.peek()